In [1]:
#import necessary libraries
import geopandas as gpd

In [5]:
buildings = gpd.read_file('Data/Southampton.gpkg')
floodzone = gpd.read_file('Data/Flood_Map_for_Planning_Rivers_and_Sea_Flood_Zone_2Polygon.shp')

In [9]:
#Convert buildings crs to match floodzone crs
buildings= buildings.to_crs(floodzone.crs)
#Ensure both shapefiles are in the same crs
print(buildings.crs)
print(floodzone.crs)

EPSG:27700
EPSG:27700
